# Digit Recognition

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Path to folder containing csv files
dataset_path = 'data/'

In [3]:
train_df = pd.read_csv(dataset_path+'train.csv')
train_data = train_df.values
train_data.shape

(42000, 785)

In [4]:
# Extract labels
train_labels = train_data[:, 0]

In [6]:
# Extract images
train_images = train_data[:,1:]
train_images.shape

(42000, 784)

### Visualize few images from the dataset